# Data Wrangling: *WeRateDogs* Twitter Data 

In [5]:
# Importing required libraries
import pandas as pd
import numpy as np
import tweepy
import requests
import re
import json
import matplotlib.pyplot as plt
import datetime
import os
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## 1- Data Gathering 

### The twitter archive and image prediction gathering

In [6]:
# Twitter archive data 
archive = pd.read_csv('twitter-archive-enhanced.csv')

# Image prediction data

image_predictions = pd.read_csv('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv', sep='\t')

In [7]:
archive.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


In [8]:
image_predictions.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


In [9]:
# Downloading and saving the image prediction data using Requests

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url)
file_name = url.split('/')[-1]
if not os.path.isfile(file_name):
    with open(file_name, 'wb') as f:
        f.write(r.content)

### Twitter API Data for the favourites and retweets counts

In [10]:
import config 
consumer_key = '7IRTUjDbsW5mAemizSNq4A94a'
consumer_secret = 'X9uGNBt8z4brGsfzSm98QUnWpMoNkYMx9WCgPeoTdRufFkAubv'
access_token = '1002855692-yfhGLsMm4mD2096j3Bd63yBIxY0e8agnbFDPKoG'
access_secret = 'kw86gw3YihmsWFLkj2u8iTdIyBFyWxXv2WXmfzPIyASIY'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

ModuleNotFoundError: No module named 'config'

In [11]:
# Experimenting to extract one tweet's id information

exp_tweet = api.get_status(archive.tweet_id[1000], tweet_mode = 'extended')
content = exp_tweet._json
content

NameError: name 'api' is not defined

In [ ]:
#Checking the keys of each tweet
content.keys()

In [ ]:
# Getting the retweet and favorite counts
content['retweet_count'] , content['favorite_count']

In [ ]:
#investigating the user information
content['user'].keys()

In [ ]:
content['user']['followers_count'], content['user']['location']

In [ ]:
# Creating a file for the additional tweets text data
if not os.path.isfile('tweet_json.txt'):
    with open ('tweet_json.txt', 'w') as file:
        for tweet_id in archive['tweet_id']:
            try:
                status = api.get_status(tweet_id, wait_on_rate_limit=True,  wait_on_rate_limit_notify=True, tweet_mode = 'extended')
                json.dump(status._json, file)
                file.write('\n')
            except Exception as e:
                print("Error on tweet id {}".format(tweet_id) + ";" + str(e))
                errors.append(tweet_id)

In [ ]:
tweets = []

with open('tweet_json.txt', 'r') as file:
    for line in file:
        tweet = json.loads(line)
        tweets.append(tweet)
        
tweets[0].keys()

In [ ]:
api_df = pd.DataFrame()

api_df['id'] = list(map(lambda tweet: tweet['id'], tweets))
api_df['retweet_count'] = list(map(lambda tweet: tweet['retweet_count'], tweets))
api_df['favorite_count'] = list(map(lambda tweet: tweet['favorite_count'], tweets))
api_df['followers_count'] = list(map(lambda tweet: tweet['user']['followers_count'], tweets))


In [ ]:
api_df.head()

In [ ]:
api_df.info()

Resources:
    

## 2. Data Assessment

That's where the inspection of our collected data sets from both the ***Quality*** and ***Tidiness*** prespectives will be conducted.

- Data quality dimensions help guide the thought process while assessing and also cleaning. The four main data quality dimensions are:

    -  Completeness: do we have all of the records that we should? Do we have missing records or not? Are there specific rows, columns, or cells missing?
    -  Validity: we have the records, but they're not valid, i.e., they don't conform to a defined schema. A schema is a defined set of rules for data. These rules can be real-world constraints (e.g. negative height is impossible) and table-specific constraints (e.g. unique key constraints in tables).
    -  Accuracy: inaccurate data is wrong data that is valid. It adheres to the defined schema, but it is still incorrect. Example: a patient's weight that is 5 lbs too heavy because the scale was faulty.
    -  Consistency: inconsistent data is both valid and accurate, but there are multiple correct ways of referring to the same thing. Consistency, i.e., a standard format, in columns that represent the same data across tables and/or within tables is desired.

##### Dimensions of the 3 datasets:

In [ ]:
print("archive: \n" + str(archive.shape) + "\n" + 'image_predictions: \n' + str(image_predictions.shape) + '\n' + 'api_df: \n' + str(api_df.shape))

### I. The Twitter *`archive`* table:

In [ ]:

archive


In [ ]:
archive.columns

In [ ]:
archive.info()

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
# Are all or most of dogs classified
print(archive.doggo.value_counts(), '\n') 
print(archive.floofer.value_counts(),'\n')
print(archive.pupper.value_counts(),'\n')
print(archive.puppo.value_counts())

In [ ]:
(archive.loc[:, 'doggo':"puppo"] != 'None') .sum()

In [ ]:
#Is the classification correct and mutually exclusive
# https://stackoverflow.com/questions/42461086/subset-pandas-dataframe-using-values-from-two-columns
nonunique_stage = archive[(archive['doggo'] != 'None') & (archive['pupper'] != 'None')]
nonunique_stage.iloc[:, -4:].head()

In [ ]:
# Checking the most frequent names
archive.name.value_counts().head(10)

In [ ]:
# Checking the name column and in relation to the text
archive.loc[:,['text', 'name']].sample(10)

In [ ]:
archive.loc[:,['text', 'name']][(archive['name'] == 'None')].sample(10)

In [ ]:
# Are all names capitalized.
archive.name.str.islower().sum()

In [ ]:
# Seems that the way the names are captured is by 
# scraping the word after the "this is" or the word "meet"
archive.loc[[1049] ,['text', 'name']]

In [ ]:
# some texts contain more than one name:
archive.loc[[1007] ,['text', 'name']]

In [ ]:
# Some names weren't successfully captured from the text
archive.loc[[391,1501] ,['text', 'name']]

In [ ]:
# Checking the ratings values
archive.loc[:, ['rating_numerator', 'rating_denominator']].describe()

In [ ]:
archive.rating_denominator.sort_values()

In [ ]:
archive.rating_numerator.sort_values()

In [ ]:
archive.source.value_counts()

### II. The *`image_predictions`* table

In [ ]:
image_predictions.head()

In [ ]:
image_predictions.sample(15)

In [ ]:
image_predictions.info()

In [ ]:
image_predictions[(image_predictions['p1_dog'] == False) & 
                  (image_predictions['p2_dog'] == False) &
                  (image_predictions['p3_dog'] == False)]

In [ ]:
image_predictions.p1.value_counts().head(15)

In [ ]:
image_predictions.p2.value_counts().head(15)

In [ ]:
image_predictions.p3.value_counts().head(15)

In [ ]:
image_predictions.loc[:,['p1_conf', 'p2_conf', 'p3_conf']].describe()

### III. The twitter *`api_df`* table

In [ ]:
api_df.head()

In [ ]:
api_df.sample(20)

In [ ]:
api_df.info()

In [ ]:
api_df.loc[:, ['retweet_count', 'favorite_count', 'followers_count']].describe()

### Data Warngling Scope of work
As per the project requirements; only original ratings (*no retweets*) that ***have images** should be included

In [ ]:
archive.columns

In [ ]:
archive.retweeted_status_id.dtypes

In [ ]:
# Seems that the way the names are captured is by 
# scraping the word after the strings "this is" or  "meet"
archive.loc[[1049] ,['text', 'name']]

## Assessment summary

### Quality aspects

#### *`archive`* table

+ **Data types(consistency issues):** 
    -  All timestamps are object type
    - All tweet_ids are integers
    - Inconsistent representation of null values as "None" strings in the (name, doggo, floofer, pupper, puppo) columns.
    - The variables (in_reply_to_status_id,  in_reply_to_user_id, 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp') will not be included in our analysis, their main use will be to identify retweets(drop retweeted_status_id with non-null values) and replies (drop in_reply_to_status_id with non-null values)to exclude them. 
    
 
- **completeness isues:** 
    -  In the name column, some names weren't successfully extracted from the text e.g. idexes 391,1501 while others contain more than one name in the text while only one was extracted e.g. index 1007.
    -  Missing entries in expanded_urls. 
    - columns (in_reply_to_status_id,  in_reply_to_user_id, 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'doggo', 'floofer', 'pupper', 'puppo') include a large number of missimg values.
    - Some tweets may not include any image; those should be deleted (note the discrepancy in the number of tweets between the `archive` dataset and the `image_prediction` one)
    - Some tweets are actually retweets and replies not original tweets that have to be deleted as per the data wrangling scope mandated by the project specification. (Note: those tweets should be removed from the three tables in hand)

- **Accuracy issues:**
    -  erroneous names extracted like the second most frequent name "the letter a". Those needs to be rendered null.
    - Incorrect and weird values of the rating_numerator which has a maximum of 1776.
    - The same holds for the rating_denominator with illogical maximum of 170

#### *`image_predictions`* table
- Non-descriptive columns' names
- Inconsistent capitalization for the prdicted breeds(p1, p2, p3 )
- In general the total number of records (2075 instead of 2356) indicates the presence of some tweets without images which we need to exclude.

#### *`api_df`* table
- In general the total number of records (2236) indicates the presence of some tweets without relevant retweet and favorite counts. 

### Tidiness aspects:

#### *`archive`* table
- values are column names(doggo, floofer, pupper, puppo); they better be combined in one column names "dog_stage"

#### *`image_predictions`* table
- values are column names(p1,p2,p3) which are all breed predictions

#### *`api_df`* table
- This isn't considered an observational unit to have its own table; so, it should be merged to the *`archive`* table

## 3. Data Cleaning

In [ ]:
# Copying the datasets 
archive_clean = archive.copy()
image_clean = image_predictions.copy()
api_clean = api_df.copy()

### Addressing the missing values in the archive dataset

**1. In the *`name`* column:**
 >   -  some names weren't successfully extracted from the text e.g. idexes 391, 1501 
     -  while others contain more than one name in the text while only one was extracted e.g. index 1007. *(This will not be addressed due as a tradeoff with keeping the dataset intact and to not jeoparadize its structure for fixing sucu a rare issue)*
     -  erroneous names extracted like the second most frequent name "the letter a". Those needs to be rendered null.
     -  Inconsistent representation of null values as "None" strings

#### Define
>-  I will firstly fix the visually detected rows manually. Dealing with the names with the string "a" value.  
-  Scrutinizing the texts that were used to extract the "a" names; i noticed a pettern in many of them that they contain the name of the dog after the word "named"; The following code will extract the name if exists and replace the value "a" with it, otherwise will set the "a" to "None".
-  Replacing the 'None' string with pythonic null values

#### Code

In [ ]:
# The visually detected rows with names not correctly extracted
archive_clean.loc[[archive_clean.index[391],archive_clean.index[1501] ], ['text', 'name']]

In [ ]:
# archive_clean.loc[archive_clean.dog_stage == "", 'dog_stage'] = np.nan
archive_clean.loc[391, 'name'] = 'Dew'
archive_clean.loc[1501, 'name'] = 'Teddy'

In [ ]:
# Many texts have the names after the word 'named'
archive_clean[archive_clean['name'] == 'a'][['text', 'name']]

In [ ]:
#before dealing with the issue in hand let's investigate some names that were erroneously extracted as the letter "a"
#archive_clean.name[1853] == 'a' while it's 'Wylie'
#archive_clean.name[1955] == 'a' while it's 'Kip'
#archive_clean.name[2034] == 'a'while it's  'Jacob'

In [ ]:
#compiling a pattern to catch the names and experimenting its validity
pattern = re.compile('(?:name(?:d)?)\s{1}(?:is\s)?([A-Za-z]+)')
re.findall(pattern, 'This is a Dasani Kingfisher from Maine. His name is Daryl. Daryl doesn''t like being swallowed by a panda. 8/10 https://t.co/jpaeu6LNmW')[0]

In [ ]:
# Crafting a function to fix the name =='a'
    # column and replace it with either the name or string 'None'
    
def fix_a_names(row):
    if row['name'] == "a" or row['name'] == 'an':
        pattern = re.compile('(?:name(?:d)?)\s{1}(?:is\s)?([A-Za-z]+)')
        try:
            new = re.findall(pattern, row['text'])[0]
            row['name'] = row['name'].replace('a', new)
            row['name'] = row['name'].replace('an', new)
        except Exception:
            new = "None"
            row['name'] = row['name'].replace('a', new)
            row['name'] = row['name'].replace('an', new)
        return row['name']
    else:
        return row['name']
archive_clean['name'] = archive_clean.apply(fix_a_names, axis=1)

Resources:
[return string with first match Regex](https://stackoverflow.com/questions/38579725/return-string-with-first-match-regex)

In [ ]:
archive_clean['name'] = archive_clean.name.replace({'None':None})

#### Test

In [ ]:
# Testing the manually fixed names
archive_clean.loc[[archive_clean.index[391],archive_clean.index[1501] ], ['text', 'name']]

In [ ]:
# Testing one of the previously named 'a' while having a name
archive_clean.iloc[[1853],list(archive_clean).index('name')]

In [ ]:
# Testing the three of the name == 'a' entries
archive_clean.loc[[archive_clean.index[1853], archive_clean.index[1955], archive_clean.index[2034] ], ['text', 'name']]

In [ ]:
# Check if there is no 'a' names
'a' in archive_clean['name'] , 'an' in archive_clean['name']

In [ ]:
# Checking the conversion on 'None' strings to Nan
archive_clean.name.isnull().sum()

In [ ]:
'None' in archive_clean['name']

In [ ]:
archive_clean.name.head(15)

**2. For the *`expanded`_url* column**
>  There are 59 missing values out of 2356. Firstly this figure is a pretty low and by little research i found out that those missing values are primarily tweets without photos that shoud not be taken into account.

#### Define
Given that the Na values are tweets with no images, dropping the missing values of this variable is an appropriate decision using the dropna method on the archive_clean dataframe

#### Code

In [ ]:
archive_clean.expanded_urls.value_counts(dropna = False)

In [ ]:
archive_clean.dropna(subset=['expanded_urls'], inplace=True)

Resources:
[How to drop a row whose particular column is empty/NaN?](https://stackoverflow.com/questions/46091924/python-how-to-drop-a-row-whose-particular-column-is-empty-nan)

#### Test

In [ ]:
archive_clean.expanded_urls.isnull().sum()

**3. For the *`doggo, floofer, pupper, puppo`* columns missing values**

#### Define 
To later fix the tidiness issue of those values disguised as variables, addressing the "None" strings better be fixed first by replacing all the "None" with the empty string "".

#### Code

In [ ]:
archive_clean.iloc[:, -4:  ].describe()

In [ ]:
archive_clean.iloc[:, -4:  ] = archive_clean.iloc[:, -4:  ].replace('None','')

#### Test

In [ ]:
# Here's the actual representation of the values of the four columns
archive_clean.iloc[:, -4:  ].describe()

In [ ]:
archive_clean.doggo.value_counts(dropna=False)

### Addressing theTidiness issues:
1. The ***`archive`*** dataset. Values represented as variables for doggo, floofer, pupper and puppo columns, all are dog stages.
2. The ***`image_predictions`*** dataset also has values represented as variables:
>    -  For  p1, p2, p3, they contain dog breed predictions.
>    -  The variables (p1_conf, p2_conf, p3_conf) representing one variable for confidence level of the predictions.
>    - Finally, the columns (p1_dog, p2_dog, p3_dog) represents a boolean variable if the predicted animal was a dog or not
    
3. The ***`api_df`*** dataset better be merged with the archive datasert

**1. The archive dataset. Values represented as variables for doggo, floofer, pupper and puppo columns, all are dog stages.**
#### Define
Addressing the tidiness issue in the `archive` dataset by combining the aforementioned 4 columns into one column through simple pandas series addition opertaion. The resulting column quality issues aill be addressed later on.

#### Code

In [ ]:
archive_clean['dog_stage'] = archive_clean.doggo + archive_clean.floofer + archive_clean.pupper + archive_clean.puppo

In [ ]:
archive_clean.drop(columns=['doggo', 'floofer', 'pupper', 'puppo'], inplace=True)

#### Test

In [ ]:
archive_clean.info()

In [ ]:
archive_clean.dog_stage.value_counts()

> The Original figures were:
    -  doggo      97 
    -  floofer    10 
    -  pupper     257
    -  puppo      30 
scattered in 4 columns

**2. The image_predictions dataset also has values represented as variables**
#### Define
The ***`image_predictions`*** dataset. Values represented as variables. 
Melting the values columns into real variable columns using the **`pd.wide_to_long`** function after renaming the columns to be more descriptive and facilitate the fuction use.

#### Code

In [ ]:
#Original names
image_clean.columns

In [ ]:
image_clean.head(1)

In [ ]:
# Renaming the dataset columns
cols = ['tweet_id', 'jpg_url', 'img_num', 
       'prediction_1', 'confidence_1', 'dog_1',
       'prediction_2', 'confidence_2', 'dog_2',
       'prediction_3', 'confidence_3', 'dog_3']
image_clean.columns = cols

In [ ]:
# Reshaping the dataframe
image_clean = pd.wide_to_long(image_clean,
                              stubnames=['prediction', 'confidence', 'dog'], 
    i=['tweet_id', 'jpg_url', 'img_num'], j='prediction_level', sep="_")\
.reset_index()

#### Test

In [ ]:
# Checking the new shape
image_clean.head(10)

In [ ]:
image_clean.info()

In [ ]:
# Checking the overall number of rows has been tripled
6225/2075

*Resources: 
-  [Pandas Melt several groups of columns into multiple target columns by name](https://stackoverflow.com/questions/38862832/pandas-melt-several-groups-of-columns-into-multiple-target-columns-by-name)
-  [pandas.wide_to_long](https://pandas.pydata.org/pandasdocs/stable/reference/api/pandas.wide_to_long.html)

**3. The** ***`api_df`*** **dataset better be merged with the `archive` datasert**
#### Define
Using the pd.merge function to merge the two datasets
### Code

In [ ]:
api_clean.info()

In [ ]:
archive_clean = pd.merge(left=archive_clean, right=api_clean, how='left', left_on='tweet_id', right_on='id')


#### Test

In [ ]:
archive_clean.drop(columns=['id'], inplace=True)
archive_clean.head()

In [ ]:
archive_clean.info()

### Addressing the Quality aspects

**1. Tweets with no photos should be removed**
#### Define
Use the `iamge_prediction` table to guide the selection and removal of tweets without photos in the `archive` table
#### Code

In [ ]:
# creating a list of tweet_ids with images "tweets_with_image" and confirming its length
tweets_with_image = list(image_clean.tweet_id.unique())
len(tweets_with_image)

In [ ]:
# confirming that all the tweets with images exist in the archive dataset
archive_clean.tweet_id.isin(tweets_with_image).sum()

In [ ]:
# Cleaning in action ;)
archive_clean = archive_clean[archive_clean.tweet_id.isin(tweets_with_image)]

#### Test

In [ ]:
# archive_clean structure and count verification
archive_clean.info()

**2. Retweets and replies removal**
### Define
In the follwing part, the following redundant columns **`(in_reply_to_status_id, in_reply_to_user_id, 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp')`** will be utilized to shed the retweet and replies from our datasets and then will be dropped.
#### Code

In [ ]:
# Throwback to the archive dataset to extract the tweets that include data in the retweet_status_id.
retweet_entries = archive_clean.retweeted_status_id.notnull()
archive_clean[retweet_entries].shape[0]

In [ ]:
# Dropping the retweets from the archive data set
archive_clean = archive_clean[~retweet_entries]

In [ ]:
# Extracting replies entries with the same method as the retweets
reply_entries = archive_clean.in_reply_to_status_id.notnull()
reply_entries.sum()

In [ ]:
# Dropping the replies from the archive datafarme
archive_clean = archive_clean[~reply_entries]

In [ ]:
# Dropping the redundant columns
archive_clean.drop(columns=['in_reply_to_status_id', 
                    'in_reply_to_user_id', 
                    'retweeted_status_id', 
                    'retweeted_status_user_id', 
                    'retweeted_status_timestamp'], inplace=True)

In [ ]:
# Now we check the image_prediction table for extra tweet ids not in the archive table
np.logical_not(image_clean.tweet_id.isin(list(archive_clean.tweet_id))).sum()

Now we detected 312 entries with ids that belong to either retweets or replies taking into consideration that this figure is actually triple the actual number of the retweets and replies combined in the image prediction dataset.

In [ ]:
# Dropping the retweets and replies ids from the image prediction dataframe
image_clean = image_clean[~np.logical_not(image_clean.tweet_id.isin(list(archive_clean.tweet_id)))]

#### Test

In [ ]:
# archive dataset counts and structure:
archive_clean.info()

In [ ]:
# Verifying the above number of records in the archive dataframe
2075 - 81 - 23

Exactly ***104 entries*** belonging to retweets and replies have been removed successfully. Triple that number should have been removed from the image prediction table

In [ ]:
image_clean.info()

In [ ]:
#Verifying the above number of records of the image_clean table
(6225 - 5913)/3

**3. Inconsistent representation of null values as "" empty-strings in the (dog_stage) column and some entries have two stages that i will separate by a hyphen for readability.**

#### Define
replacing the empty strings in the dog_stage column of the archive dataset with pythonic null values and separating the stacked stages with a hyphen by simple pandas assignment operation
#### Code

In [ ]:
# Checking the values before cleaning
archive_clean.dog_stage.value_counts(dropna=False)

In [ ]:
# Fixing the empty string issue for more accurate representation of vlues
archive_clean.loc[archive_clean.dog_stage == "", 'dog_stage'] = np.nan

In [ ]:
# Separating the stacked stages
archive_clean.loc[archive_clean.dog_stage == 'doggopupper', 'dog_stage'] = 'doggo-pupper'
archive_clean.loc[archive_clean.dog_stage == 'doggopuppo', 'dog_stage'] = 'doggo-puppo'
archive_clean.loc[archive_clean.dog_stage == 'doggofloofer', 'dog_stage'] = 'doggo-floofer'

#### Test

In [ ]:
# Verifying the cleanes version
archive_clean.dog_stage.value_counts(dropna=False)

**4. Rating numerator and rating denominator inaccurate and dubious values**
#### Define
Firstly, i set a definition for what dubious and inaccurate values actually are.
> -  For the denominator; any value below or above 10 is suspected, i will get to know how many values are there and slice their records from the dataset to closely investigate their pertinent text.
> -  For the Numerator, i suspected values below 6 and above 15 and will go through the same way of slicing their relevant tweets and investigate thier texts.
> -  Finally, i will address some of them manually as they are considered one off occurrences and try to fix the others programmatically if nececssary

### Code

In [ ]:
# Throwback for the rating figures
archive_clean.loc[:, ['rating_numerator', 'rating_denominator']].describe()

In [ ]:
# Deeper look into the numerator values with their count
archive_clean.rating_numerator.value_counts(dropna = False)

In [ ]:
# A closer insight into the value counts of the denominator
archive_clean.rating_denominator.value_counts(dropna=False)

In [ ]:
# Setting  slicers for the data based on the set boundaries
odd_numerator = np.logical_or(archive_clean.rating_numerator > 15, archive_clean.rating_numerator <= 5)
odd_denominator = np.logical_or(archive_clean.rating_denominator > 10, archive_clean.rating_denominator < 10)
odd_numerator.sum(), odd_denominator.sum()

There are **17 odd denominator** values and **101 numerators**, let's see what the slicing of the dataframe will result in as some tweets may share oddities on both figures

In [ ]:
# Slicing the archive dataset based on the previous boolean slicers
odd_ratings = archive_clean.loc[np.logical_or(odd_denominator, odd_numerator), ['text', 'rating_numerator', 'rating_denominator']]
odd_ratings.shape[0]

It is evident that there some odd values on both sides of the rating figures with both of them resulting in **103 records** collectively

In [ ]:
pd.set_option('display.max_rows', 110)
odd_ratings

**These are the problems that will be addressed:**
- Note; this part could have been done in the assessment section, but i opted to place it here because it's diving deep into the two specific variables of the rating_denominator and rating_numerator
- ***Group (1) Fractions***: 
* After Vetting the records, i found out that the numerators with fractions that contain decimal points aren't captuerd correctly as the digits after the decimal point is the only captured part. This can be addressed programmatically.
>  -  Fractions examples: indexes (667 , 733 , 1664)

- ***Group (2) Miscelleneous reasons***: 
* Moreover, there're 10 observations that are considered special cases to be handled manually.
>  -  #491: daily activity "24/7" not a rating so to be set to Null value
>  -  #2018: "420/10" and not a dog but a person's photo , to be deleted 
>  -  #1849: two ratings for two dogs (5/10, 8/10) will take the average
>  -  #1466: two ratings for two dogs  (10/10, 5/10) will take the average
>  -  #946:  an oulier yet a valid one as per the image rating "1776/10", to be deleted
>  -  #1034: 14/10 instead of 9/11; the later is a date.
>  -  #1130: group of 4 dogs out of 20 with a rating of 13/10 each
>  -  #1167: the rating is 11/10 not 50/50
>  -  #1616: 10/10 not 7/11 
>  -  #2276: 9/10 not 1/2

- ***Group (3) Packs of dogs with collective ratings***: 
- In the follwing tweets, the images display groups of dogs (after opening the link in the text), the rating here is collective for all of them, but after counting the number of dogs in each photo, i discovered that the denominator is equal to **Number of dogs** multiplied by **10**.
- In accordance, we have two possible choices to deal with them;
    -  Either to replace the ratings with value for each individual dog after the division of both the numerator and denominator by the number of dogs. 
    -  Or we can leave it as it's beacause the quotient of dividing the numerator by the denominator will be the same.   
- I will go for the first choice for the sake of getting numerator and denominator figures that conform with the specific schema outlined in the project description.
>  -  #410: a pack of 7 dogs with overall rating of 84/70  -- 12/10
>  -  #870: a pack of 10 dogs with overall rating of 165/150 --> 11/10
>  -  #1085: a pack of 17 dogs with overall rating of 204/170 --> 12/10
>  -  #1193: a pack of 9 dogs with overall rating of 99/90 --> 11/10
>  -  #1219: a pack of 8 dogs with overall rating of 80/80 --> 10/10
>  -  #1239: a pack of 5 dogs with overall rating of 45/50 --> 9/10
>  -  #1314: a pack of 5 dogs with overall rating of 60/50 --> 12/10
>  -  #1396: a pack of 4 dogs with overall rating of 44/40 --> 11/10
>  -  #1314: a pack of 5 dogs with overall rating of 60/50 --> 12/10
>  -  #1589: a pack of 11 dogs with overall rating of 121/110 --> 11/10
>  -  #1730: a pack of 12 dogs with overall rating of 144/120 --> 12/10
>  -  #1793: a pack of 5 dogs with overall rating of 88/80 --> 12/10

#### Group (1) Code

In [ ]:
 # Compiling a regex to capture the decimal as well as whole numbers and testing it on some sample text from the dataframe
num_p = re.compile('(\d+\.?\d?\d?)\/(\d{1,3})')
print(re.findall(num_p, 'This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948'))
re.findall(num_p, 'This is Logan, the Chow who lived. He solemnly swears he''s up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS')

In [ ]:
# Using the previously compiled pattern to extract the nominator in more robust way across the whole datafram
archive_clean['rating_numerator'] = archive_clean.text.str.extract('(\d+\.?\d?\d?)\/\d{1,3}',expand = False).astype('float')

#### Group (1) Testing

In [ ]:
# Verifying that the ratings with fractions were correctly captured e.g. indeixec (667 , 733 , 1664):
archive_clean.loc[[667 , 733 , 1664], ['text', 'rating_numerator', 'rating_denominator']]

#### Group (2) Code

In [ ]:
#index - 491: daily activity "24/7" not a rating
archive_clean.loc[[491], ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Setting this tweet ratings to null
archive_clean.loc[491, ['rating_numerator', 'rating_denominator']] =  [np.nan , np.nan] 

In [ ]:
#Index - 946: An oulier yet a valid one as per the image rating "1776/10", to be deleted not to bias fututre analysis
archive_clean.loc[[946], ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Dropping this tweet from the datafram
archive_clean.drop(946, inplace=True)

In [ ]:
# Index - 1034: 14/10 instead of 9/11; the later is a date.
archive_clean.loc[[1034], ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Correcting the rating
archive_clean.loc[1034, ['rating_numerator', 'rating_denominator']] =  [14 , 10] 

In [ ]:
# Index - 1130: group of 4 dogs out of 20 with a rating of 13/10 each
archive_clean.loc[[1130], ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Replace with the accurate values
archive_clean.loc[1130, ['rating_numerator', 'rating_denominator']] =  [13 , 10]

In [ ]:
#Index - 1167: the rating is 11/10 not 50/50
archive_clean.loc[[1167], ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Correcting the rating
archive_clean.loc[1167, ['rating_numerator', 'rating_denominator']] =  [11 , 10]

In [ ]:
# Index - 1466: two ratings for two dogs (10/10, 5/10) will take the average
archive_clean.loc[[1466], ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Setting the rating to the average:
archive_clean.loc[1466, ['rating_numerator', 'rating_denominator']] =  [7.5 , 10]

In [ ]:
# Index - 1616: 10/10 not 7/11
archive_clean.loc[[1616], ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Correcting the values
archive_clean.loc[1616, ['rating_numerator', 'rating_denominator']] =  [10 , 10]

In [ ]:
# Index - 1849: two ratings for two dogs (5/10, 8/10) will take the average
archive_clean.loc[[1849], ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Setting rating to the average
archive_clean.loc[1849, ['rating_numerator', 'rating_denominator']] =  [6.5 , 10]

In [ ]:
# Index - 2018: "420/10" and not a dog but a person's photo , to be deleted 
archive_clean.loc[[2018] ,['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Giving its extreme numerator value and after checking the image that is not for a dog, i opted to exclude this tweet
archive_clean.drop(2018, inplace=True)

In [ ]:
#Index - 2276: 9/10 not 1/2
archive_clean.loc[[2276] ,['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Correcting the ratings values
archive_clean.loc[2276, ['rating_numerator', 'rating_denominator']] =  [9 , 10]

#### Group(2) Test

In [ ]:
# index - 491: Confirming that this tweet rating was reolaced by null values
archive_clean.loc[[491], ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Index - 946: verifying the deletion
archive_clean.loc[945:948, ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Index - 1034: confriming the correction
archive_clean.loc[[1034], ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Index - 1130: confriming the correction
archive_clean.loc[[1130], ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Index - 1167: confriming the correction
archive_clean.loc[[1167], ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Index - 1466: set to the average
archive_clean.loc[[1466], ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Index - 1616: 10/10 
archive_clean.loc[[1616], ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Index - 1616: set to average
archive_clean.loc[[1849], ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Index 2018 is deleted
archive_clean.loc[2017:2020, ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
#Index - 2276: 9/10 not 1/2
archive_clean.loc[[2276] ,['text', 'rating_numerator', 'rating_denominator']]

#### Group (3) Code
> -Via simple pandas series arithmetic operations 

In [ ]:
indices = [410, 870, 1085, 1193, 1219, 1239, 1314, 1396, 1589, 1730,1793]
archive_clean.loc[indices, ['text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Calculating the dogs counts in the tweets featuring many dogs
dogs_count = archive_clean.rating_denominator[archive_clean.rating_numerator >= 40]/10
dogs_count

In [ ]:
# Performing the Calculations
archive_clean.loc[archive_clean.rating_numerator >= 40, ['rating_numerator', 'rating_denominator']] = [archive_clean.rating_numerator[archive_clean.rating_numerator >= 40]/dogs_count , 10]

#### Group(3) Test

In [ ]:
# Confirming the switch to the individual ratings figures for consistency
archive_clean.loc[indices, ['text', 'rating_numerator', 'rating_denominator']]

## Test

In [ ]:
#Testing for overall ratings figures
archive_clean.loc[:, ['rating_numerator', 'rating_denominator']].describe()

In [ ]:
archive_clean.rating_numerator.value_counts()

In [ ]:
archive_clean.rating_denominator.value_counts()

- With the removal of just dubious tweets and fixing the faulty figures, the rating_numerator and rating_denominator are now ready for producing reliable analysis

**5. Fixing datatypes** 
#### Define
- The `dog_stage` in the `archive` table from string to categorical
- The timestamp column type into datetime object using the function pd.to_datetime
- The `prediction_level` in the `image_prediction` table from integer to categorical
- The `tweet_id` in both table from integer to string.
#### Code

In [ ]:
# Datatypes in the archive table
archive_clean.tweet_id = archive_clean.tweet_id.astype('str')
archive_clean.dog_stage = archive_clean.dog_stage.astype('category')
archive_clean['timestamp'] = pd.to_datetime(archive_clean.timestamp)

In [ ]:
# Datatypes in the image prediction table
image_clean.tweet_id = image_clean.tweet_id.astype('str')
image_clean.prediction_level = image_clean.prediction_level.astype('category')

#### Test

In [ ]:
archive_clean.info()

In [ ]:
image_clean.info()

**6. Inconsistent entries capitalization of the `prediction` column in the `image_prediction` dataframe**
#### Define
Applying the string method `.capitalize` on this column
#### Code

In [ ]:
image_clean.prediction = image_clean.prediction.str.capitalize()

#### Test

In [ ]:
image_clean.prediction.value_counts().head(20)

## 4- Saving the Cleaned data Locally

In [ ]:
archive_clean.to_csv('twitter_archive_master.csv', index=False, encoding='utf-8')
image_clean.to_csv('image_predictions_tidy.csv', index=False, encoding='utf-8')

## 5- Data Insights `(Analysis and Visualization)`

In [ ]:
arc_df = pd.read_csv('twitter_archive_master.csv')
predictions_df = pd.read_csv('image_predictions_tidy.csv')

In [ ]:
# Checking the characteristics of the twitter archive data after reloading
arc_df.info()

In [ ]:
# Checking the characteristics of the image prediction data after reloading
predictions_df.info()

**It's clear that the datatypes have been changed during the save-reload process, restoring them by running the following code**

In [ ]:
# Datatypes in the arc_df
arc_df.tweet_id = arc_df.tweet_id.astype('str')
arc_df.dog_stage = arc_df.dog_stage.astype('category')
arc_df['timestamp'] = pd.to_datetime(arc_df.timestamp)

In [ ]:
# Datatypes in the image predictions_df
predictions_df.tweet_id = predictions_df.tweet_id.astype('str')
predictions_df.prediction_level = predictions_df.prediction_level.astype('category')

### Exploratory visualizations

In [ ]:
sns.set_context(context='notebook')

In [ ]:
# Investigating Patterns and relationships in arc_df table
pd.plotting.scatter_matrix(arc_df.iloc[:,1:], figsize=(15,15));

From the scatter-matrix we can spot a positive relationship between the retweet_count and favorite_count. Also, it seems that there is some correlation between the rating numerator and both retweet_count and favorite_count.

**1. Correlation matrix**

In [ ]:
arc_df.iloc[:, 1:].corr(method='pearson')

- As observed from the scatter-matrix, there is a strong positive linear relationship retweet_count and favorite_count `(corr_coeff = 0.9297)`. 
- There's also a moderate positive relationship between the rating_numerator and favorite_count`(corr_coeff = 0.396)`. A weaker postive relationship between the rating_numerator and retweet_count is indicated by the `correlation coefficient = 0.299`

### Retweet Counts & Favorite Counts 

**2. Scatter plot between the retweet_count and favorite_count**

In [ ]:
sns.set_context('talk')
def r2(x,y):
    return stats.pearsonr(x,y)[0]**2
sns.jointplot(arc_df.favorite_count, arc_df.retweet_count, kind="reg", stat_func=r2 )
plt.tick_params(axis="both", labelsize=12);
plt.savefig("retweet_count_favorite_count_corr.png")

Plotting the relation between retweet_count and favorite_count shows with the pearson r^2 of 0.86 illustrates the strong positive relationship between these two variavbles 

Resources:
[Seaborn implot with equation and R2 text](https://stackoverflow.com/questions/25579227/seaborn-implot-with-equation-and-r2-text)

In [ ]:
arc_df.loc[:, ['retweet_count', 'favorite_count']].describe()

- I created another dataframe (arc) in which the timestamp column is the index and grouped it by monthly means for smoother plotting of variables like the favorite counts and retweet counts. I such way, i removed the noise of plotting timestamp data.

In [ ]:
# for plotting
arc = pd.read_csv('twitter_archive_master.csv', index_col = 'timestamp', parse_dates=True)

In [ ]:
arc.index = arc.index.date
arc.index = pd.to_datetime(arc.index, infer_datetime_format=True)
arc = arc.resample('M').mean()

In [ ]:
arc.loc[:,['rating_numerator','retweet_count','favorite_count','followers_count']].describe()

**3. Average retweet_count and favorite_count trend over time span of the data**

In [ ]:
sns.set_context('talk')
arc['favorite_count'].plot(style='k', figsize=(12,8), label='Favorite count');
arc.retweet_count.plot(style='b',label='Retweet count');
plt.legend()
plt.yscale("log")
plt.xlabel("Date");plt.ylabel("Average Count");
plt.title('Favorites & Retweets over time')
plt.tight_layout()
plt.savefig("favorite_Retweet_count_trend.png")

**4. Tweets count trend over the dataset time period**

In [ ]:
arc_timeseries = arc_df.copy()
arc_timeseries = arc_timeseries.set_index('timestamp')
arc_timeseries.sort_index()
master_timeseries = arc_timeseries.reset_index().merge(predictions_df[predictions_df.prediction_level == 1],
         on='tweet_id', how='left').set_index(arc_timeseries.index)

In [ ]:
sns.set_context('notebook')
master_timeseries.groupby([(master_timeseries.index.year), (master_timeseries.index.month)]).tweet_id.count().plot(kind='line',color='r',linestyle='--',
marker='D',figsize=(12,4));
mn=np.mean(master_timeseries.groupby([(master_timeseries.index.year), (master_timeseries.index.month)]).count()['tweet_id'])
plt.axhline(y=mn, color='b', linestyle='--', label='Out of rating')
plt.xlabel("Date");plt.ylabel("Tweet Count");
plt.title('Tweets Monthly count')
plt.savefig("tweets_monthly_count_trend.png")

**5. retweet_count and favorite_count for tweets featuring dogs and those not featuring dogs**

In [ ]:
master_timeseries.groupby(['dog']).retweet_count.describe()

In [ ]:
sns.set_context('poster')
ax = sns.boxplot(x='dog', y='retweet_count', data=master_timeseries, showfliers=False)
plt.savefig("boxplots_retweet_count_dogs_not_dogs.png")

In [ ]:
ax = sns.boxplot(x='dog', y='retweet_count',  
                 data=master_timeseries[master_timeseries.retweet_count<7500])

In [ ]:
master_timeseries.groupby(['dog']).favorite_count.describe()

In [ ]:
sns.set_context('talk')
ax = sns.boxplot(x='dog', y='favorite_count', data=master_timeseries, showfliers=False)
plt.setp(ax.get_xticklabels(), rotation=45)
plt.savefig("boxplots_favorite_count_dogs_not_dogs.png")

### Rating and their relationship with other variables
**6. Rating patterns over time**

In [ ]:
# creating a copy of the arc_df with timestapm as index
sns.set_context('poster')
arc_timeseries = arc_df.copy()
arc_timeseries.index = arc_timeseries['timestamp']
arc_timeseries.drop(columns=['timestamp'], inplace=True)
arc_timeseries.sort_index(inplace=True)
arc_timeseries.groupby([(arc_timeseries.index.year),
                       (arc_timeseries.index.month)]).rating_numerator.mean().plot(style='-ro', figsize=(12,8),label='Total average rating')
plt.axhline(y=10.0, color='b', linestyle='--', label='Out of rating')
plt.xlabel("Date rated");
plt.ylabel("Rating out of 10");
plt.legend();
plt.savefig("Average_Rating_pattern.png")

**7. Rating distribution for images featuring dogs and images with no dogs**

In [ ]:
sns.set_context('notebook')
plt.subplots(figsize=(6,4))
sns.boxplot(master_timeseries.index.year, master_timeseries.rating_numerator, hue=master_timeseries.dog, palette="Set1");
plt.legend(loc=8)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.2)  ,ncol=3, fancybox=True, shadow=True)
plt.xlabel("Rating Year"); plt.ylabel("Rating");
plt.tight_layout()
plt.savefig("Rating_per_year_for_dogs_no_dogs.png")

**Resources**: [How to keep index when using pandas merge](https://stackoverflow.com/questions/11976503/how-to-keep-index-when-using-pandas-merge)

**8. Dog Stage distribution**

In [ ]:
sns.set_context('poster')
ordered_stages = arc_df['dog_stage'].value_counts().head(4).index
sns.countplot(data = arc_df, x = 'dog_stage', order=ordered_stages ,orient = 'h')
plt.title('Dog stages Distribution',fontsize=14)
plt.tight_layout()
plt.savefig("Dog_stage_distribution.png");

In [ ]:
sns.set_context('talk')
ax = sns.boxplot(x='dog_stage', y='favorite_count', showfliers=False, width = .70,
                 data=master_timeseries)
sns.set(rc={'figure.figsize':(8.0,16.0)})
plt.savefig("Dog_stage_favorite_count.png")

In [ ]:
sns.set_context('talk')
ax = sns.boxplot(x='dog_stage', y='retweet_count', showfliers=False, width = .70,
                 data=master_timeseries)
sns.set(rc={'figure.figsize':(15.5, 7.5)});
plt.savefig("Dog_stage_retweet_count.png")

Given the large amount of missing data concerning  this variable, This plot shows that the pupper stage is the most prevalent stage. This may be attributed to the fact that younger puppies are usually more attractive. 

**9. Glimpse on Breeds** 

In [ ]:
sns.set_context('talk')
master_timeseries[master_timeseries.dog == True].prediction.value_counts()[12::-1].plot(kind='barh')
plt.xlabel('tweets_count')
plt.ylabel('Dog Breed');
plt.savefig("Top_Breeds_Distribution.png")

In [ ]:
sns.set_context('talk')
master_timeseries[master_timeseries.dog == True].groupby(['prediction']).retweet_count.sum().sort_values(ascending=False)[12::-1].plot(kind='barh');
plt.xlabel('Retweets_count')
plt.ylabel('Dog Breed');
plt.savefig("Top_Breeds_Retweet_Count.png")

In [ ]:
sns.set_context('talk')
master_timeseries[master_timeseries.dog == True].groupby(['prediction']).favorite_count.sum().sort_values(ascending=False)[12::-1].plot(kind='barh');
plt.xlabel('favorites_count')
plt.ylabel('Dog Breed');
plt.savefig("Top_Breeds_Favorite_count.png")

**10. Followers Insight**

In [ ]:
plt.subplots(figsize=(12, 8))
plt.plot(arc.followers_count)
plt.xlabel('\nDate (YYYY-MM)')
plt.ylabel('Number of Followers\n');
plt.savefig('followers_decline.png')

I will stop here, yet definitly there are a ton of insights to glean and analyse...!